In [1]:
import pandas as pd
import spacy

### Load data

In [26]:
data= pd.read_csv('../Dataset/train.csv')
data.lang.unique()

array(['bn', 'de', 'en', 'es', 'fa', 'fr', 'hi', 'it', 'pt', 'sv', 'uk',
       'zh'], dtype=object)

In [27]:
data = data[data['lang']=='it']

### Information Extraction Pipeline

In [3]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
tokenizer = AutoTokenizer.from_pretrained("Davlan/xlm-roberta-base-ner-hrl")
model = AutoModelForTokenClassification.from_pretrained("Davlan/xlm-roberta-base-ner-hrl")
nlp = pipeline("ner", model=model, tokenizer=tokenizer)


In [28]:
#import the pipeline class


from InformationExtraction import InformationExtractionPipeline


# example spacy extractor function
NER = spacy.load("en_core_web_lg")
def tag_extraction_from_spacy(sen, model = NER):
    
    annotated = model(sen)
    extracted_names = [word.text for word in annotated.ents 
                       if word.label_=='PERSON' or word.label_=='ORG'or word.label_=='GPE']
    
    
    return extracted_names


def tag_extraction_from_LM(sen, model = nlp):
    
    ner_results = model(sen)
    extracted_names = []
    for idx in range(len(ner_results)):
        if ner_results[idx]['entity'][0] == 'B':
            start = ner_results[idx]['start']
            end = ner_results[idx]['end']
            j = idx+1
            while j < len(ner_results):
                if ner_results[j]['entity'][0] == 'B':
                    break
                elif ner_results[j]['entity'][0] == 'I':
                    end = ner_results[j]['end']
                j+=1
            idx = j
        
            extracted_names.append(sen[start:end].strip())
    
    
    return extracted_names


# example extractor function that uses training labels 
sent_to_tag = dict(zip(data['sent'],data['labels']))
def tag_extraction_from_tags(sent, sent_to_tag=sent_to_tag):

    tags = sent_to_tag[sent]
    sentsWithtags = [(s,t) for s,t in zip(sent.split(),tags.split())]
    entity_list = []
    for i,item in enumerate(sentsWithtags):
        if 'B-' in item[1]:
            j = i
            entity = []
            while j<len(sentsWithtags):
                if sentsWithtags[j][1] =='O':
                    break
                entity.append(sentsWithtags[j][0])
                j+=1
            i = j
         
            entity_list.append(" ".join(entity))
            
    

    return entity_list




In [34]:
#create pipline object:
#param: extractor: an entity extractor function that returns all the entities from a sentence
#param: max_sen: define the number of sentences to be added for each detected entity
#param: lang: define language. needed for wikipedia api
#param: saveJson: whether to save extracted informaton as json file. Saves time if needed to run the pipeline again
#param: loadJson: if you have saved a json file and want to use it
#param: jsonPath: define saved json file path

infoPipeline = InformationExtractionPipeline(extractor = tag_extraction_from_LM, max_sen = 2, 
                                         lang = 'it', loadJson = True, jsonPath='wiki-info-it-train.json',
                                         saveJson=True, saveJsonpath='wiki-info-it-train.json')

In [ ]:
#call pipline and provide list of sentences as argument

augmented = infoPipeline(data['sent'].values.tolist())

 25%|█████████▋                            | 4213/16578 [10:50<14:14, 14.46it/s]

In [ ]:
# infoPipeline.information_extraction(data['sent'].values.tolist()[1])


### Info Percentage

In [31]:
data['augmented_sen'] = augmented
temp = data[data['sent']!=data['augmented_sen']]


In [32]:
info_percent = temp.shape[0]/data.shape[0]
print(f"Info Percentage: {info_percent*100:.2f}%")

Info Percentage: 60.05%


### Save Augmented Data

In [33]:
data.to_csv('../Dataset/train-wiki-it-LM.csv',index=False)